# Import libraries

This code imports libraries from Jar files

In [1]:
import sys
import os
import time
import jpype
import jpype.imports

try:
    sdk_gradle_home = os.environ["ATALA_PRISM_JARS"]
except KeyError:
    print("ERROR: `ATALA_PRISM_JARS` variable is not set.")
    print("Please, set it to the directory with Atala PRISM SDK dependencies JARs.")
    sys.exit(1)

# 'io' is a standard Python module
# we have to add a new domain to be able to import from Java implicitly
# give it the 'sdk' name for this purpose
try:
    jpype.imports.registerDomain('sdk', alias='io')
except ImportError as err:
    print(err.msg)
    sys.exit(ERROR)
jpype.startJVM(
    classpath=[
        os.path.join(sdk_gradle_home, f) for f in os.listdir(sdk_gradle_home)
            if f.endswith('.jar')
    ]
)

from sdk.iohk.atala.prism.protos import *
from sdk.iohk.atala.prism.api.node import *
from sdk.iohk.atala.prism.api.models import *
from sdk.iohk.atala.prism.api import *
from sdk.iohk.atala.prism.crypto.derivation import *
from sdk.iohk.atala.prism.crypto.keys import *
from sdk.iohk.atala.prism.identity import *
from kotlinx.serialization.json import *

KeyGenerator = KeyGenerator.INSTANCE
KeyDerivation = KeyDerivation.INSTANCE
MasterKeyUsage = MasterKeyUsage.INSTANCE
IssuingKeyUsage = IssuingKeyUsage.INSTANCE
RevocationKeyUsage = RevocationKeyUsage.INSTANCE
AuthenticationKeyUsage = AuthenticationKeyUsage.INSTANCE
AtalaOperationStatus = AtalaOperationStatus.INSTANCE

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


# Utilities

**Please note:** publishing a `DID` to Cardano ledger might take about 10 minutes. That's where `waitUntilConfirmed` could be used to wait for given operation to be confirmed and applied by the PRISM Node.

In [2]:
def wait_until_confirmed(node_api: NodePublicApi, operation_id: AtalaOperationId):
    """Waits until operation is confirmed by Cardano network

    Confirmation doesn't necessarily mean that operation was applied.
    For example, it could be rejected because of an incorrect signature or other reasons.

    :param node_api: Atala PRISM Node API object
    :type node_api: NodePublicApi
    :param operation_id: Atala PRISM operation ID
    :type operation_id: AtalaOperationId
    """
    operation_status = int(node_api.getOperationStatus(operation_id).join())
    while operation_status != AtalaOperationStatus.getCONFIRMED_AND_APPLIED() \
        and operation_status != AtalaOperationStatus.getCONFIRMED_AND_REJECTED():
        print(f"Current operation status: {AtalaOperationStatus.asString(operation_status)}")
        time.sleep(1)
        operation_status = int(node_api.getOperationStatus(operation_id).join())

def prepare_keys_from_mnemonic(mnemonic: MnemonicCode, password: str):
    """Creates a map of potentially useful keys out of a mnemonic code

    :param mnemonic: Mnemonic to generate keys from
    :type mnemonic: MnemonicCode
    :param password: Password for seed generation
    :type password: str
    :return: Map of keys
    :rtype: dict<String, ECKeyPair>
    """
    did_key_map = {}
    seed = KeyDerivation.binarySeed(mnemonic, password)
    did_key_map[PrismDid.getDEFAULT_MASTER_KEY_ID()] = KeyGenerator.deriveKeyFromFullPath(seed, 0, MasterKeyUsage, 0)
    did_key_map[PrismDid.getDEFAULT_ISSUING_KEY_ID()] = KeyGenerator.deriveKeyFromFullPath(seed, 0, IssuingKeyUsage, 0)
    did_key_map[PrismDid.getDEFAULT_REVOCATION_KEY_ID()] = KeyGenerator.deriveKeyFromFullPath(seed, 0, RevocationKeyUsage, 0)
    return did_key_map

# Backend clients

At last, let's create the client for the Node backend service, which is involved in this tutorial

In [3]:
environment = "master.atalaprism.io"
node_auth_api = NodeAuthApiAsyncImpl(GrpcOptions("https", environment, 50053))

## Issuer: Generate Identity

The **Issuer's** goal in this tutorial is to issue a credential to **Holder**, and eventually, revoke it.

In order to do that, some preparation steps need to be done:
1. Generate a **Decentralized Identifier (DID)** for **Issuer**.
2. Publish the **Issuer's DID** to **Cardano**, which is a necessary step for anyone willing to issue credentials.

During preparation, set up a project that includes **Atala PRISM SDK** and proceed to do the **Issuer** steps.

**NOTE:** Extensive knowledge or understanding of **DIDs** is not required to complete this tutorial. Check the official [DID specifications](https://w3c-ccg.github.io/did-spec/) or [Decentralized identifiers on Wiki](https://en.wikipedia.org/wiki/Decentralized_identifiers) for reference.

### Generating a DID

Generating a **DID** requires associating a public key. This example generates a public key from a random mnemonic code and then publishes the **DID** to **Cardano** by invoking the **Node** service:

In [4]:
print("Issuer: Generates and registers a DID")
issuer_keys = prepare_keys_from_mnemonic(KeyDerivation.randomMnemonicCode(), "passphrase")
issuer_unpublished_did = PrismDid.buildLongFormFromMasterPublicKey(
    issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPublicKey()
)
issuer_did = issuer_unpublished_did.asCanonical()
node_payload_generator = NodePayloadGenerator(
    issuer_unpublished_did,
    {PrismDid.getDEFAULT_MASTER_KEY_ID(): issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey()}
)
issuer_create_did_info = node_payload_generator.createDid(PrismDid.getDEFAULT_MASTER_KEY_ID())
issuer_create_did_operation_id = node_auth_api.createDid(
    issuer_create_did_info.getPayload(),
    issuer_unpublished_did,
    PrismDid.getDEFAULT_MASTER_KEY_ID()
).join()

print(f"""
- Issuer sent a request to create a new DID to PRISM Node.
- The transaction can take up to 10 minutes to be confirmed by the Cardano network.
- Operation identifier: {issuer_create_did_operation_id.hexValue()}
""")
create_did_operation_result = wait_until_confirmed(node_auth_api, issuer_create_did_operation_id)

print(f"- DID with id {issuer_did} is created")

Issuer: Generates and registers a DID

- Issuer sent a request to create a new DID to PRISM Node.
- The transaction can take up to 10 minutes to be confirmed by the Cardano network.
- Operation identifier: a24c74d1dda4dfb4e1850896f418957ed6bb25ce9854389d22c55fe4ee967bab

Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current 

## Holder: Generate Identity

The **Holder's** goal in this tutorial is to receive a credential from **Issuer**. In order to do that, you will need to generate an **unpublished DID** to identify **Holder**. In general, it's a good practice for **Holders** to generate a new DID for each certificate, in order to improve privacy.

Please note, that in this simplistic example, credential is immediately available to **Verifier** for verification, but in some more realistic scenario, it would require a communication channel between **Holder** and **Verifier** in order to exchange a credential.

### Generating Holder DID
The **Holder DID** is generated similarly to how the **Issuer DID** was generated. The main difference is that there is no proof in **Cardano** about the existence of such **DID**.

In [ ]:
holder_keys = prepare_keys_from_mnemonic(KeyDerivation.randomMnemonicCode(), "secret")
holder_unpublished_did = PrismDid.buildLongFormFromMasterPublicKey(
    holder_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPublicKey()
)
print(f"Holder: DID generated: {holder_unpublished_did}")

## Issuer: Issue Credential

Now, **Issuer** is able to issue a credential to **Holder**.

### Update Issuer's DID Document
Before we issue a certificate, we have to update issuer's **DID Document** with issuer's **Issuing Keys**.

In [ ]:
# Generator should contain the issuing key so let's create a new instance of it with this key inside
issuing_node_payload_generator = NodePayloadGenerator(
    issuer_unpublished_did,
    {
        PrismDid.getDEFAULT_MASTER_KEY_ID(): issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey(),
        PrismDid.getDEFAULT_ISSUING_KEY_ID(): issuer_keys[PrismDid.getDEFAULT_ISSUING_KEY_ID()].getPrivateKey()
    }
)

issuing_key_info = PrismKeyInformation(
    DidPublicKey(
        PrismDid.getDEFAULT_ISSUING_KEY_ID(),
        IssuingKeyUsage,
        issuer_keys[PrismDid.getDEFAULT_ISSUING_KEY_ID()].getPublicKey()
    ), None, None
)

add_issuing_key_did_info = issuing_node_payload_generator.updateDid(
    issuer_create_did_info.getOperationHash(),
    PrismDid.getDEFAULT_MASTER_KEY_ID(),
    [issuing_key_info], []
)

add_issuing_key_operation_id = node_auth_api.updateDid(
    add_issuing_key_did_info.getPayload(),
    issuer_did,
    PrismDid.getDEFAULT_MASTER_KEY_ID(),
    issuer_create_did_info.getOperationHash(),
    [issuing_key_info], []
).join()
print(f"""
Issuer: Add issuing key, the transaction can take up to 10 minutes to be confirmed by the Cardano network
- IssuerDID = {issuer_did}
- Add issuing key to DID operation identifier = {add_issuing_key_operation_id.hexValue()}
""")
update_did_operation_result = wait_until_confirmed(node_auth_api, add_issuing_key_operation_id)

**NOTE:** Update DID operation requires hash of the previous operation performed for that DID, in this case, it was the createDid operation.

### Prepare credential

Now we are ready. Let's create a simple certificate for **Holder** and sign it with the **Issuer's** key:

In [ ]:
# Issuer generates a credential to Holder identified by its unpublished DID
credential_claim = CredentialClaim(
    holder_unpublished_did,
    JsonObject(
        {
            "name": JsonLiteral("Jose Lopez Portillo", True),
            "certificate": JsonLiteral("Certificate of PRISM SDK Python tutorial completion", True)
        }
    )
)
issue_credential_info = issuing_node_payload_generator.issueCredentials(
    PrismDid.getDEFAULT_ISSUING_KEY_ID(),
    [credential_claim]
)

**NOTE:** In our protocol, issuing a credential involves creating batches of signed credentials to reduce the time and costs of publishing them. A batch is created even if we publish a single credential.

## Publish the credential to Cardano

Once we have prepared the batch, we can invoke **Atala PRISM Node** to publish it to **Cardano**:

In [ ]:
issue_credential_batch_operation_id = node_auth_api.issueCredentials(
    issue_credential_info.getPayload(),
    issuer_did,
    PrismDid.getDEFAULT_ISSUING_KEY_ID(),
    issue_credential_info.getMerkleRoot()
).join()

issue_credential_batch_operation_result = \
    wait_until_confirmed(node_auth_api, issue_credential_batch_operation_id)

**Print out some details of the publish credential operation:**

In [ ]:
holder_signed_credential = issue_credential_info.getCredentialsAndProofs()[0].getSignedCredential()
holder_credential_merkle_proof = issue_credential_info.getCredentialsAndProofs()[0].getInclusionProof()

print(f"""
Issuer [{issuer_did}] issued new credentials for the holder [{holder_unpublished_did.asCanonical()}].
- issueCredentialBatch operation identifier: {issue_credential_batch_operation_id.hexValue()}
- Credential content: {holder_signed_credential.getContent()}
- Signed credential: {holder_signed_credential.getCanonicalForm()}
- Inclusion proof (encoded): {holder_credential_merkle_proof.encode()}
- Batch id: {issue_credential_info.getBatchId()}"""
)

## Verifier: Verify Valid Credential

**Verifier**, who owns credential claim, can easily verify the validity of the credential. Verifying the credential requires
a single method call, which in this case succeeds because the credential is valid:

In [ ]:
# Verifier, who owns credentialClam, can easily verify the validity of the credentials
credential_verification_result = node_auth_api.verify(
    holder_signed_credential,
    holder_credential_merkle_proof
).join()

verification_errors = credential_verification_result.getVerificationErrors()
assert list(verification_errors) == [], "VerificationErrors should be empty"

**NOTE:** In general, **Verifier** would have its DID, generated and published to the **Cardano** in the same way as **Issuer**.

## Issuer: Revoke Credential

This section illustrates how **Issuer** can revoke a credential. The API allows to:
1. Revoke a single credential.
2. Revoke all credentials involved in a batch.
3. Revoke many credentials from the same batch.

This example shows how to revoke a single credential, but, the other options are available by just switching the method arguments.

### Revoke a single credential

Before revoking credentials, we need to update the **DID Document** with the new **Revocation Key** that will be used to revoke credentials.
After this, just generate the revocation operation and get both operations posted in **Cardano** by the **Node**:


In [ ]:
# Generator should contain the issuing key so let's create a new instance of it with this key inside
issuer_node_payload_generator = NodePayloadGenerator(
    issuer_unpublished_did,
    {
        PrismDid.getDEFAULT_MASTER_KEY_ID(): issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey(),
        PrismDid.getDEFAULT_ISSUING_KEY_ID(): issuer_keys[PrismDid.getDEFAULT_ISSUING_KEY_ID()].getPrivateKey(),
        PrismDid.getDEFAULT_REVOCATION_KEY_ID(): issuer_keys[PrismDid.getDEFAULT_REVOCATION_KEY_ID()].getPrivateKey()
    }
)

revocation_key_info = PrismKeyInformation(
    DidPublicKey(
        PrismDid.getDEFAULT_REVOCATION_KEY_ID(),
        RevocationKeyUsage,
        issuer_keys[PrismDid.getDEFAULT_REVOCATION_KEY_ID()].getPublicKey()
    ), None, None
)

add_revocation_key_did_info = issuer_node_payload_generator.updateDid(
    add_issuing_key_did_info.getOperationHash(),
    PrismDid.getDEFAULT_MASTER_KEY_ID(),
    [revocation_key_info], []
)

add_revocation_key_operation_id = node_auth_api.updateDid(
    add_revocation_key_did_info.getPayload(),
    issuer_did,
    PrismDid.getDEFAULT_MASTER_KEY_ID(),
    add_issuing_key_did_info.getOperationHash(),
    [revocation_key_info], []
).join()
wait_until_confirmed(node_auth_api, add_revocation_key_operation_id)

# Revoke credentials
revoke_credentials_info = issuer_node_payload_generator.revokeCredentials(
    PrismDid.getDEFAULT_REVOCATION_KEY_ID(),
    issue_credential_info.getOperationHash(),
    issue_credential_info.getBatchId().getId(),
    [holder_signed_credential.hash()]
)
revoke_credentials_operation_id = node_auth_api.revokeCredentials(
    revoke_credentials_info.getPayload(),
    issuer_did,
    PrismDid.getDEFAULT_REVOCATION_KEY_ID(),
    issue_credential_info.getOperationHash(),
    issue_credential_info.getBatchId().getId(),
    [holder_signed_credential.hash()]
).join()
print(f"""
Issuer: Asked PRISM Node to revoke credentials. The transaction can take up to 10 minutes to be confirmed by the Cardano network
- addRevocationKey operation identifier: {add_revocation_key_operation_id.hexValue()}
- revokeCredentials operation identifier: {revoke_credentials_operation_id.hexValue()}"""
)
wait_until_confirmed(node_auth_api, revoke_credentials_operation_id)
print("Credentials susccessfully revoked")

## Verifier: Verify Revoked Credential

Analogously to the last verification section, **Verifier** will verify the credential again. However, since it was revoked in the previous step, this time verification **is supposed to fail**.

The operations take some minutes to be applied by **Cardano**, once the revoke operation is confirmed, query the revocation time and run the verification again, this time, an exception is thrown explaining that the credential is revoked:

In [ ]:
print("Verifier: Checking the credential validity again, expect an error explaining that the credential is revoked")
credential_revocation_time = node_auth_api.getCredentialRevocationTime(
    issue_credential_info.getBatchId().getId(),
    holder_signed_credential.hash()
).join()

# Verifier checks the credential validity (which fails)
new_credential_verification_result = node_auth_api.verify(
    holder_signed_credential,
    holder_credential_merkle_proof
).join()
verification_errors = new_credential_verification_result.getVerificationErrors()

assert credential_revocation_time.getLedgerData().getTimestampInfo() == \
verification_errors[0].getTimestamp(), \
"CredentialWasRevokedOn error is expected"

## Issuer: Deactivate DID

This section illustrates how the owner of the identity can deactivate the DID by removing all public keys from the corresponding DID Document.


In [ ]:
print("Issuer: deactivating DID by removing all public keys from the document")
deactivate_issuer_did_info = issuer_node_payload_generator.deactivateDid(
    add_revocation_key_did_info.getOperationHash(),
    PrismDid.getDEFAULT_MASTER_KEY_ID()
)
deactivate_issuer_operation_id = node_auth_api.deactivateDID(
    deactivate_issuer_did_info.getPayload(),
    issuer_did,
    PrismDid.getDEFAULT_MASTER_KEY_ID(),
    add_revocation_key_did_info.getOperationHash()
).join()
wait_until_confirmed(node_auth_api, deactivate_issuer_operation_id)

## Next steps

**That's it, congratulations on completing Node Integration Tutorial!**

By now, one should have a decent understanding how to replicate the work done on the [Atala PRISM Interactive Demo Website](https://atalaprism.io).
